In [2]:
# NOTEBOOK IMPORTS
import os, glob, warnings, shutil
import numpy as np
from datetime import datetime

# IMAGE IMPORTS
from PIL import Image

# GIS IMPORTS
from affine import Affine
import pandas as pd

# PLOTTING IMPORTS
import matplotlib.pyplot as plt

# NN IMPORTS
import ultralytics

# CUSTOM UTILITIES
from IndexUtils import * 

Image.MAX_IMAGE_PIXELS = 933120000
warnings.filterwarnings("ignore")

In [3]:
data_dir = r"C:\Users\fhacesga\Desktop\FIRMsDigitizing\data\\"
input_folder  = r"D:\FloodChange\AAA_HistoricalDownload\Files\\"
infer_folder  = r"C:\Users\fhacesga\Desktop\FIRMsDigitizing\data\RLNN\CurrentPredictions050124\\"


In [4]:
patterns = ["IND", "_1."]
index_files = [file for pattern in patterns for file in glob.glob(input_folder + "\\*" + pattern + "*")]

all_files = glob.glob(input_folder + "\\*")
tiles     = list(set(all_files) - set(index_files))

# REMOVE WORLD FILES
tiles = [tile for tile in tiles if tile[-1] != 'w']

In [14]:
model = ultralytics.YOLO(f"{data_dir}/RLNN/best.pt").to("cuda")

outputs = []
batch_size = 1

bboxs = {}

for tile in tqdm(tiles):
    # try:
    results = model(tile, imgsz=(512, 512), verbose=False, conf=0.7)
    bboxs[os.path.basename(tile).split('.')[0]] = results[0].boxes.xyxyn.cpu().numpy()
    for result in results:
        fn = os.path.join(infer_folder, os.path.basename(result.path))
        result.save(filename=os.path.join(infer_folder, os.path.basename(result.path)))
    '''
    except:
        print(f"Error in {tile}")
        continue'''

model = model.to("cpu")
torch.cuda.empty_cache()

  0%|          | 0/1264 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [15]:
bboxs

{'485486A_4': array([[  0.0072603,     0.14256,     0.93585,       0.963]], dtype=float32),
 '4802960010D': array([[   0.029067,   0.0058188,     0.89714,     0.99572]], dtype=float32),
 '480035A_21': array([[   0.070365,    0.001226,     0.91632,     0.95612]], dtype=float32),
 '4800450014A': array([[  0.0045616,           0,     0.90007,           1]], dtype=float32),
 '480423_4': array([[   0.038649,    0.011239,     0.76993,     0.98003]], dtype=float32),
 '480035A_20': array([[   0.057723,   0.0029948,     0.93023,     0.95381]], dtype=float32)}

In [ ]:
def findBounds(image_fn, model=None, 
        model_weights=f"{data_dir}RLNN/best.pt",
        creation_params=None,
        device="cpu",
        ):
    
    if creation_params is None:
        target_size = 512
        original_shapes = []

        # COCO DATASET PARAMS
        category_labels = {
            0 : "County",
            1 : "Tile",
            2 : "Box",
            3 : "Legend"
        }

        categories=[0, 1]

    input_folder = os.path.dirname(os.path.abspath(image_fn))

    # Initialize model
    if model is None:
        model = ultralytics.YOLO(model_weights).to("cpu")

    model = model.to(device)
    
    results = model(image_fn, imgsz=target_size)

    return results